In [10]:
# Data processing
import pandas as pd
import numpy as np

# Machine learning
import shap

# Serialising
import joblib
import base64
import pickle
import io

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

from tabulate import tabulate

In [2]:
# Load the string from the Object Store
qb = QuantBook()
key = "Chocolate"

model_data_str = qb.ObjectStore.Read(key + ".pto")
print(model_data_str)

# Convert the string back to a dictionary
model_data = json.loads(model_data_str)
model = joblib.load(io.BytesIO(base64.b64decode(model_data["model"])))

In [3]:
def VisualiseFeatures():

    # Extract model data
    features = model_data["features"]

    # Unpack the features and associated colors
    categories = list(features.keys())
    heights = list(features.values())
    colors = ['red' if height < 0 else 'green' for height in heights]

    # Create the bar graph
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=categories, y=heights, palette=colors)
    plt.xticks(rotation=90)

    # Add labels to the axes
    plt.xlabel('Factors')
    plt.ylabel('Coefficients')

    # Add baseline
    plt.axhline(y=0, linewidth=1, color='k')

    # Add colored text
    for i, height in enumerate(heights):
        label_color = 'black' if height < 0 else 'white'
        ax.text(i, 0.01, f'{height:.2f}', ha='center', va='bottom', color=label_color, fontsize=10)

    # Add title
    plt.title('Factor Coefficients for the Autotrading Model')

    plt.show()

VisualiseFeatures()

In [4]:
def VisualiseError():

    # Extract train std and model MAE
    std = model_data["data_stats"]["train"]["std"]
    ml_mae = model_data["data_stats"]["ML"]["error"]
    mfm_mae = model_data["data_stats"]["MFM"]["error"]

    # Unpack the features and associated colors
    categories = ["Guess at Mean", "ML Model", "Multi-Factor Model"]
    heights = [std/2, ml_mae ,mfm_mae]

    # Create the bar graph
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=categories, y=heights)

    # Add labels to the axes
    plt.xlabel('Factors')
    plt.ylabel('Coefficients')

    # Add baseline
    plt.axhline(y=0, linewidth=1, color='k')

    # Add colored text
    for i, height in enumerate(heights):
        label_color = 'black' if height < 0 else 'white'
        ax.text(i, 0.01, f'{height:.2f}', ha='center', va='bottom', color=label_color, fontsize=10)

    # Add title
    plt.title('Error of Model Compared to Baselines')

    plt.show()

VisualiseError()


In [5]:
def VisualiseDataBehaviour():

    # Create data for two variables
    data_mean = model_data["data_stats"]["train"]["mean"]
    data_std = model_data["data_stats"]["train"]["std"]
    simulation = np.random.normal(data_mean, data_std, len(model_data["data_stats"]["ML"]["preds"]))

    ml_preds = model_data["data_stats"]["ML"]["preds"]
    mfm_preds = model_data["data_stats"]["MFM"]["preds"]    

    # Create a DataFrame to hold the data
    df = pd.DataFrame({'Data': simulation, 'ML Predicitons': ml_preds, 'mfm_preds': mfm_preds})

    # Create a box and whisker plot
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df,  orient='h', showfliers=False)
    plt.ylabel('Values')
    plt.title('Box and Whisker Plot of Two Variables')

    plt.show()

VisualiseDataBehaviour()

In [9]:
def PrintSetup():

    print(tabulate([
        ['Start Date', model_data["setup_stats"]["start_date"]], 
        ['End Date', model_data["setup_stats"]["end_date"]],
        ['Start Cash', model_data["setup_stats"]["start_cash"]],
        ['Portfolio Size', model_data["setup_stats"]["num_long"]], 
        ['Rebalance Days', model_data["setup_stats"]["rebalance_days"]]],
        tablefmt='psql' ,headers=['Statistic', 'Value']))

    # Capture all table data
    data = [
        ['Total Return', model_data["backtest_stats"]["total_return"]],
        ['Annualised Return', model_data["backtest_stats"]["annual_return"]], 
        ['Sharpe Ratio', model_data["backtest_stats"]["sharpe_ratio"]], 
        ['Max Drawdown', model_data["backtest_stats"]["drawdown"]], 
        ['Alpha', model_data["backtest_stats"]["alpha"]],
        ['Beta', model_data["backtest_stats"]["beta"]], 
        ['Win Rate', model_data["backtest_stats"]["drawdown"]], 
        ['Information Ratio', model_data["backtest_stats"]["ir"]]
    ]

    # Print the table
    table = tabulate(data, tablefmt='psql' ,headers=['Statistic', 'Value'])
    print(table)


PrintSetup()

In [11]:
def PrintBacktestStats():

    # Capture all table data
    data = [
        ['Total Return', model_data["backtest_stats"]["total_return"]],
        ['Annualised Return', model_data["backtest_stats"]["annual_return"]], 
        ['Sharpe Ratio', model_data["backtest_stats"]["sharpe_ratio"]], 
        ['Max Drawdown', model_data["backtest_stats"]["drawdown"]], 
        ['Alpha', model_data["backtest_stats"]["alpha"]],
        ['Beta', model_data["backtest_stats"]["beta"]], 
        ['Win Rate', model_data["backtest_stats"]["drawdown"]], 
        ['Information Ratio', model_data["backtest_stats"]["ir"]]
    ]

    # Print the table
    table = tabulate(data, tablefmt='psql' ,headers=['Statistic', 'Value'])
    print(table)

PrintBacktestStats()

In [12]:
def VisualiseInvestments(n):

    # Create the bar graph
    plt.figure(figsize=(10, 6))

    top_investments = {A:N for (A,N) in [x for x in model_data['investments'].items()][:n]} 

    ax = sns.barplot(x=list(top_investments.values()), y=list(top_investments.keys()))

    # Add labels to the axes
    plt.xlabel('Years Invested')
    plt.ylabel('Stock')

    # Add title
    plt.title(f'Top {n} Investments of the Autotrading Model')

    # displaying chart 
    plt.show() 

VisualiseInvestments(25)